# Import Libraries

In [1]:
#https://github.com/tanishqgautam/Drone-Image-Semantic-Segmentation/blob/main/semantic-segmentation-pytorch.ipynb

In [2]:
!pip install segmentation-models-pytorch #installam bibliotecile pe colab
!pip install -U albumentations
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
from google.colab import drive
drive.mount('/content/gdrive')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [3]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import torchvision
import torch.nn.functional as F
from torch.autograd import Variable

from PIL import Image
import cv2
import albumentations as A

import time
import os

from tqdm.notebook import tqdm

!pip install segmentation-models-pytorch
!pip install torchsummary

from torchsummary import summary
import segmentation_models_pytorch as smp

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Using cuda device


# Preprocessing

In [4]:
# IMAGE_PATH = '../input/semantic-drone-dataset/dataset/semantic_drone_dataset/original_images/'
# MASK_PATH = '../input/semantic-drone-dataset/dataset/semantic_drone_dataset/label_images_semantic/'

In [5]:
# n_classes = 23 

# def create_df():
#     name = []
#     for dirname, _, filenames in os.walk(IMAGE_PATH):
#         for filename in filenames:
#             name.append(filename.split('.')[0])
    
#     return pd.DataFrame({'id': name}, index = np.arange(0, len(name)))

# df = create_df()
# print('Total Images: ', len(df))

In [6]:
# #split data
# X_trainval, X_test = train_test_split(df['id'].values, test_size=0.1, random_state=19)
# X_train, X_val = train_test_split(X_trainval, test_size=0.15, random_state=19)

# print('Train Size   : ', len(X_train))
# print('Val Size     : ', len(X_val))
# print('Test Size    : ', len(X_test))

In [7]:
# img = Image.open(IMAGE_PATH + df['id'][100] + '.jpg')
# mask = Image.open(MASK_PATH + df['id'][100] + '.png')
# print('Image Size', np.asarray(img).shape)
# print('Mask Size', np.asarray(mask).shape)


# plt.imshow(img)
# plt.imshow(mask, alpha=0.6)
# plt.title('Picture with Mask Appplied')
# plt.show()

In [8]:
os.chdir('/content/gdrive/MyDrive/date_dizertatie')
base_path = '/content/gdrive/MyDrive/date_dizertatie'


#pregatim cuda
print("Numar cudauri" + str(torch.cuda.device_count()))
print("Nume placa video " + str(torch.cuda.get_device_name(0)))



Numar cudauri1
Nume placa video NVIDIA A100-SXM4-40GB


In [9]:
x_train_dir = os.path.join(base_path, 'train_graz/x')
y_train_dir = os.path.join(base_path, 'train_graz/y')

x_val_dir = os.path.join(base_path, 'valid_graz/x')
y_val_dir = os.path.join(base_path, 'valid_graz/y')

x_test_dir = os.path.join(base_path, 'test_graz/x')
y_test_dir = os.path.join(base_path, 'test_graz/y')

# Dataset

In [10]:
class DroneDataset(Dataset):
    
    def __init__(self, img_path, mask_path, mean, std, transform=None, patch=False):
        self.img_path = img_path
        self.mask_path = mask_path
        self.transform = transform
        self.patches = patch
        self.mean = mean
        self.std = std

        self.ids = sorted(os.listdir(img_path))
        self.images_fps = [os.path.join(img_path, image_id) for image_id in self.ids]
        self.mask_ids = [i.replace("jpg", "png") for i in self.ids]#originalele sunt jpg dar mastile png
        self.masks_fps = [os.path.join(mask_path, mask_id) for mask_id in self.mask_ids]
        
    def __len__(self):
        return len(os.listdir(self.img_path))
    
    def __getitem__(self, idx):
        img = cv2.imread(self.images_fps[idx])
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.masks_fps[idx], cv2.IMREAD_GRAYSCALE)
        
        if self.transform is not None:
            aug = self.transform(image=img, mask=mask)
            img = Image.fromarray(aug['image'])
            mask = aug['mask']
        
        if self.transform is None:
            img = Image.fromarray(img)
        
        t = T.Compose([T.ToTensor(), T.Normalize(self.mean, self.std)])
        img = t(img)
        mask = torch.from_numpy(mask).long()
        
        if self.patches:
            img, mask = self.tiles(img, mask)
            
        return img, mask
    
    def tiles(self, img, mask):

        img_patches = img.unfold(1, 512, 512).unfold(2, 768, 768) 
        img_patches  = img_patches.contiguous().view(3,-1, 512, 768) 
        img_patches = img_patches.permute(1,0,2,3)
        
        mask_patches = mask.unfold(0, 512, 512).unfold(1, 768, 768)
        mask_patches = mask_patches.contiguous().view(-1, 512, 768)
        
        return img_patches, mask_patches

In [18]:
mean=[0.485, 0.456, 0.406]
std=[0.229, 0.224, 0.225]

t_train = A.Compose([A.Resize(704, 1056, interpolation=cv2.INTER_NEAREST), A.HorizontalFlip(), A.VerticalFlip(), 
                     A.GridDistortion(p=0.2), A.RandomBrightnessContrast((0,0.5),(0,0.5)),
                     A.GaussNoise()])

t_val = A.Compose([A.Resize(704, 1056, interpolation=cv2.INTER_NEAREST), A.HorizontalFlip(),
                   A.GridDistortion(p=0.2)])

#datasets
train_set = DroneDataset(x_train_dir, y_train_dir, mean, std, t_train, patch=False)
val_set = DroneDataset(x_val_dir, y_val_dir, mean, std, t_val, patch=False)

#dataloader
batch_size= 3 

train_loader = DataLoader(train_set, batch_size=8, shuffle=True)
val_loader = DataLoader(val_set, batch_size=2, shuffle=False)               

# Model

In [19]:
model = smp.Unet('mobilenet_v2', encoder_weights='imagenet', classes=23, activation=None, encoder_depth=5, decoder_channels=[256, 128, 64, 32, 16])

In [20]:
#model

# Training

In [16]:
import segmentation_models_pytorch.utils as smp_utils
DEVICE = 'cuda'

loss = smp_utils.losses.DiceLoss()
metrics = [
    smp_utils.metrics.IoU(threshold=0.5),
]

optimizer = torch.optim.Adam([ 
    dict(params=model.parameters(), lr=0.0001),
])

# create epoch runners 
# it is a simple loop of iterating over dataloader`s samples
train_epoch = smp.utils.train.TrainEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    optimizer=optimizer,
    device=DEVICE,
    verbose=True,
)

valid_epoch = smp.utils.train.ValidEpoch(
    model, 
    loss=loss, 
    metrics=metrics, 
    device=DEVICE,
    verbose=True,
)



In [21]:
max_score = 0

for i in range(0, 40):
    
    print('\nEpoch: {}'.format(i))
    train_logs = train_epoch.run(train_loader)
    valid_logs = valid_epoch.run(val_loader)
    
    # do something (save model, change lr, etc.)
    if max_score < valid_logs['iou_score']: #scoatem validarea ca nu merge, rezolvam mai tarziu
        max_score = valid_logs['iou_score']
        torch.save(model, './best_model.pth')
        print('Model saved!')
        
    if i == 25:
        optimizer.param_groups[0]['lr'] = 1e-5
        print('Decrease decoder learning rate to 1e-5!')


Epoch: 0
train:   0%|          | 0/32 [00:09<?, ?it/s]


RuntimeError: ignored